In [5]:
# convert point cloud as (480, 640) numpy deph image to pcd file

import numpy as np
import open3d as o3d


for i in range(1, 211):
    depth_image = np.load(f"data\depth_image.npy ({i}).npy")
    pcd = o3d.geometry.PointCloud()
    
    x1 = np.linspace(0, depth_image.shape[1] - 1, depth_image.shape[1])
    y1 = np.linspace(0, depth_image.shape[0] - 1, depth_image.shape[0])
    x1, y1 = np.meshgrid(x1, y1)
    z1 = depth_image.flatten()
    
    pcd.points = o3d.utility.Vector3dVector(np.array([x1.flatten(), y1.flatten(), z1]).T)
    o3d.io.write_point_cloud(f"data\point_cloud ({i}).pcd", pcd)


In [7]:
%pip install tqdm


     ---------------------------------------- 78.3/78.3 KB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\manu\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [1]:
import open3d as o3d
import numpy as np
import tqdm

# Lista de archivos de nubes de puntos
point_cloud_files = [f"data\point_cloud ({i}).pcd" for i in range(1, 210)]

# Crear una nube de puntos vacía para fusionar todas las nubes
merged_cloud = o3d.geometry.PointCloud()

print("Merging point clouds...")

for file in tqdm.tqdm(point_cloud_files, desc="Merging point clouds"):
    cloud = o3d.io.read_point_cloud(file)
    merged_cloud += cloud
    # progressbar in console
    
    
print("Point clouds merged!")
print("Voxelizing and filtering...")
# Filtrado de Voxel Grid
voxel_down_pcd = merged_cloud.voxel_down_sample(voxel_size=0.01)


print("Saving merged point cloud...")
# Filtrado estadístico
cl, ind = voxel_down_pcd.remove_statistical_outlier(nb_neighbors=50, std_ratio=1.0)
filtered_cloud = voxel_down_pcd.select_by_index(ind)

# Segmentación de planos
plane_model, inliers = filtered_cloud.segment_plane(distance_threshold=0.01,
                                                    ransac_n=3,
                                                    num_iterations=1000)
inlier_cloud = filtered_cloud.select_by_index(inliers)

# Extracción de clusters
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug):
    labels = np.array(filtered_cloud.cluster_dbscan(eps=0.02, min_points=10, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

# Reconstrucción de la superficie
# Compute normals
filtered_cloud.estimate_normals()

# Ball pivoting
radii = [0.005, 0.01, 0.02, 0.04]
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    filtered_cloud, o3d.utility.DoubleVector(radii))

# Guardar el modelo
# o3d.io.write_triangle_mesh("path/to/your/reconstructed_model.ply", rec_mesh)

# Visualización
o3d.visualization.draw_geometries([filtered_cloud, rec_mesh])


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Merging point clouds...


Merging point clouds: 100%|██████████| 209/209 [00:16<00:00, 12.76it/s]


Point clouds merged!
Voxelizing and filtering...
Saving merged point cloud...
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 0
point cloud has 0 clusters
[Open3D WARNING] [SimpleShaderForTriangleMesh] Binding failed with empty triangle mesh.
[Open3D WARNING] [SimpleShaderForTriangleMesh] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForTriangleMesh] Something is wrong in compiling or binding.
[Open3D WARNING] [SimpleShaderForTriangleMesh] Binding failed with empty triangle mesh.
[Open3D WARNING] [SimpleShaderForTriangleMesh] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForTriangleMesh] Something is wrong in compiling or binding.
[Open3D WARNING] [SimpleShaderForTriangleMesh] Binding failed with empty triangle mesh.
[Open3D WARNING] [SimpleShaderForTriangleMesh] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForTriangleMesh] So